In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers as kl
import random
from pickle import load

In [2]:
with open('data/winning_data.pkl', 'rb') as f:
    f.seek(0)
    winning_data = load(f)
with open('data/indextocard.pkl', 'rb') as f:
    f.seek(0)
    card_index = load(f)
with open('data/cardtoindex.pkl', 'rb') as f:
    f.seek(0)
    card_dict = load(f)

In [3]:
print(len(winning_data))

44373


In [4]:
def datasetup(draft):
    cards = 0
    alldrafted = np.zeros(343, np.int8)
    maindeck = np.zeros(343, np.int8)
    for key,value in draft.items():
        if key.startswith("main_"):
            cards = cards + int(value)
            card = key.split("main_")[1]
            maindeck[int(card_dict[card])] = value
        if key.startswith("draft_"):
            card = key.split("draft_")[1]
            alldrafted[int(card_dict[card])] = value
    return cards, alldrafted, maindeck

In [5]:
def getremovedcards(deck, draft):
    removedlist = []
    draftlist = []
    for index, value in enumerate(deck):
        for i in range(0, value):
            array = np.zeros(343, np.int8)
            array[index] = 1
            removedlist.append(array)
            draftlist.append(draft)
    removedlist = np.array(removedlist) 
    draftlist = np.array(draftlist) 
    return removedlist, draftlist

In [6]:
def removerandomcards(deck):
  copy_deck = np.zeros(343)
  for index, value in enumerate(deck):
    if value > 0 and random.random() > 0.25:
      copy_deck[index] = copy_deck[index] + random.randint(0, value)
  return copy_deck

In [7]:
def printcards(deck):
    for index, card in enumerate(deck):
        if card > 0:
            print(card_index[index] + ", " + str(card))

In [8]:
def specialround(deck, draft):
  copydeck = deck.copy()
  for index, value in enumerate(copydeck):
    if value < 0.5:
      copydeck[index] = 0
    else:
      copydeck[index] = min(np.ceil(value), draft[index])
  return copydeck

In [9]:
def totalcards(deck):
  x = 0
  for i in deck:
    x += i
  return x

In [9]:
def calcaccuracy(pred, main):
  diff = 0
  total = 0
  for index, value in enumerate(pred):
    if value != main[index]:
      diff += abs(value - main[index])
    total += main[index]
  return 1 - diff/total

In [10]:
def exporttodraft(deckexport):
  draft = np.zeros(343)
  maindeck = np.zeros(343)
  includemain = True
  for index, value in enumerate(deckexport):
    if value.startswith('Sideboard'):
      includemain = False
    split = value.split(" ", 1)
    if len(split) == 2:
      name = split[1].split("(", 1)
      card_name = name[0].strip()
      if card_name == "Mountain" or card_name == "Island" or card_name == "Plains" or card_name == "Swamp" or card_name == "Forest":
        continue
      cardindex = card_dict[card_name]
      draft[cardindex] = draft[cardindex] + int(split[0])
      if includemain:
        maindeck[cardindex] = maindeck[cardindex] + int(split[0])
  return draft, maindeck

In [11]:
cardcount, alldrafted, removedmain = datasetup(winning_data[1])
remo = removerandomcards(removedmain)
printcards(remo)
print("---------------")
printcards(removedmain)
print(removedmain.shape)
concat = np.concatenate((removedmain, remo))
print(concat.shape)

Extus, Oriq Overlord, 1.0
Silverquill Pledgemage, 1.0
Star Pupil, 2.0
Unwilling Ingredient, 1.0
---------------
Closing Statement, 1
Combat Professor, 1
Dramatic Finale, 1
Extus, Oriq Overlord, 1
Eyetwitch, 1
Hall of Oracles, 1
Hunt for Specimens, 1
Owlin Shieldmage, 1
Rise of Extus, 3
Silverquill Apprentice, 1
Silverquill Campus, 1
Silverquill Pledgemage, 1
Star Pupil, 5
Stonerise Spirit, 2
Study Break, 1
Tenured Inkcaster, 2
Unwilling Ingredient, 1
(343,)
(686,)


In [12]:
npp = np.load('data/' + str(1) + '.npy')
print(npp[1][0])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 0 2 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0]


In [13]:
class DataGenerator(keras.utils.Sequence):
    """Generates data for Keras"""

    def __init__(self, start, end, batch_size, shuffle=False):
        """Initialization"""
        self.data_indexes = np.arange(start, end)
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        """Denotes the number of batches per epoch"""
        return int(np.floor(len(self.data_indexes) / self.batch_size))

    def __getitem__(self, index):
        """Generate one batch of data"""

        indexes = self.data_indexes[index*self.batch_size:(index+1)*self.batch_size]

        X = np.empty((self.batch_size, 343))
        y = np.empty((self.batch_size, 343))

        # Generate data
        for i, ID in enumerate(indexes):
            npp = np.load('data/' + str(ID) + '.npy')
            y[i] = npp[1][0]
            X[i] = npp[0][0]

        return X, y

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle:
            np.random.shuffle(self.data_indexes)

In [14]:
model = keras.Sequential()

model.add(kl.Dense(343, input_shape=(343, ), activation = 'relu'))
model.add(kl.BatchNormalization())
model.add(kl.Dense(343, activation = 'relu'))
model.add(kl.BatchNormalization())
model.add(kl.Dense(343, activation = 'relu'))
model.add(kl.BatchNormalization())
model.add(kl.Dense(343, activation = 'relu'))
model.add(kl.BatchNormalization())
model.add(kl.Dense(343, activation = 'relu'))
model.add(kl.BatchNormalization())
model.add(kl.Dense(343, activation = 'relu'))
model.add(kl.BatchNormalization())
model.add(kl.Dense(343, activation = 'relu'))

# Used to further train a saved model
# model = keras.models.load_model("MainDeckModel.h5")

print(model.summary(90))

metric = keras.metrics.MeanAbsoluteError(name="mean_absolute_error", dtype=None)
model.compile(loss='mean_squared_error',
              optimizer='adam', metrics=[metric])

data_gen = DataGenerator(0, 38400, 32, shuffle=True)
valid_gen = DataGenerator(38400, 44373, 32)

my_callbacks = [
    keras.callbacks.EarlyStopping(patience=5),
    keras.callbacks.ModelCheckpoint(filepath="MainDeckModel.h5",
                                    save_best_only=True),
]

model.fit(data_gen, steps_per_epoch=1200, epochs=50, validation_data=valid_gen, validation_steps=186,
          callbacks=my_callbacks, shuffle=False)

Model: "sequential"
__________________________________________________________________________________________
Layer (type)                            Output Shape                        Param #       
dense (Dense)                           (None, 343)                         117992        
__________________________________________________________________________________________
batch_normalization (BatchNormalization (None, 343)                         1372          
__________________________________________________________________________________________
dense_1 (Dense)                         (None, 343)                         117992        
__________________________________________________________________________________________
batch_normalization_1 (BatchNormalizati (None, 343)                         1372          
__________________________________________________________________________________________
dense_2 (Dense)                         (None, 343)                   

KeyboardInterrupt: 

In [15]:
model = keras.models.load_model("MainDeckModel.h5")
valid_gen = DataGenerator(38400, 44373, 32)
result = model.predict(valid_gen)

In [16]:
totalacc = 0
for index, value in enumerate(result):
  nppp = np.load('data/' + str(index + 38400) + '.npy')
  totalacc += calcaccuracy(specialround(value, nppp[0][0]), nppp[1][0])
print(totalacc/len(result))

0.6733529238565147
